# Statistical Testing for Metabolomics

In this module we will review common statistical tests performed on metabolomics data. 

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, mannwhitneyu
from sklearn.preprocessing import StandardScaler # for preprocesssing data before PCA
from sklearn.decomposition import PCA

In [4]:
# demo data are at https://github.com/shuzhao-li/khipu/tree/main/testdata/
# Have a look via pd
ecoli = pd.read_table("../Datasets/ecoli_pos.tsv", index_col=0, sep="\t")
ecoli.head()

,mz,rtime,12C_Ecoli_20220321_004,12C_Ecoli_20220321_004_20220322095030,12C_Ecoli_20220321_004_20220322130235,13C_Ecoli_20220321_004,13C_Ecoli_20220321_004_20220322132355,13C_Ecoli_20220321_004_20220322101150
id_number,,,,,,,,
F1,61.9278,40.94,42543,0,0,43369,60144,57714
F2,95.9735,125.06,2079328,984349,1314348,1707804,1299125,1159692
F3,115.9802,11.42,3362695,3228128,4076328,3201740,3942341,3326312
F4,115.9860,5.83,2035714,1739145,1978764,1977817,2040421,1642861
F5,115.9860,27.96,182603,219669,248607,199092,185630,185255


In [5]:
# t-test of each row between 12C and 13C samples
# data are log2 transformed in this test to have more normal distribution; +1 to avoid log2(0)
def ttest(row):
    t,p = ttest_ind(np.log2(row[3:6]+1), np.log2(row[6:9]+1))
    return p

pvalues_featurelist = ecoli.apply(ttest, axis=1)


In [6]:
# sort p-values
pvalues_featurelist = pvalues_featurelist.sort_values()
pvalues_featurelist.head(10)

id_number
F1039    4.028376e-13
F2201    3.386336e-11
F2955    4.471926e-11
F1417    1.217689e-10
F2675    2.118523e-10
F2784    2.630405e-10
F2663    8.330401e-10
F2930    1.497729e-09
F2874    2.037131e-09
F3182    2.161996e-09
dtype: float64

In [7]:
# how many features have p < 0.001
pvalues_featurelist[pvalues_featurelist<0.001].shape

(73,)

In [8]:
# in many situations you will need to perform some for of multiple testing correction.

# bonferroni correction is most straightforward, simply multiply the p-values by the number of p-values

pvalues_featurelist_bonferroni = pvalues_featurelist * len(pvalues_featurelist)
pvalues_featurelist_bonferroni[pvalues_featurelist_bonferroni<0.001].shape

(26,)

In [9]:
# FDR correction rather than bonferroni often will preserve statistical power better than bonferroni.
# note that we must remove any zero values from the p-value list for these procedures.

from statsmodels.stats.multitest import fdrcorrection

significant, q_vals = fdrcorrection([x for x in pvalues_featurelist if x > 0], 0.001)
print("\n".join([str(x) for x in sorted(q_vals)][:10]))

# by default this performs a benjamini-hochberg correction, other versions of the correction
# can be performed using options passed to fdrcorrection.

1.4248366846058156e-09
5.272400732087582e-08
5.272400732087582e-08
1.0767415443653224e-07
1.4986430227295902e-07
1.5506237324837282e-07
4.2092325342867044e-07
6.621835489965146e-07
7.646978620707893e-07
7.646978620707893e-07


In [10]:
# alternatively, a non-parametric test could be used. 

def mwutest(row):
    t,p = mannwhitneyu(np.log2(row[3:6]+1), np.log2(row[6:9]+1))
    return p

pvalues_featurelist_nonpara = ecoli.apply(mwutest, axis=1)

In [38]:
# sort p-values
pvalues_featurelist_nonpara = pvalues_featurelist_nonpara.sort_values()
pvalues_featurelist_nonpara.head(10)

id_number
F1495    0.106583
F2706    0.106583
F2707    0.106583
F3024    0.106583
F3022    0.106583
F1508    0.106583
F2711    0.106583
F1496    0.106583
F1843    0.106583
F3180    0.106583
dtype: float64

In [1]:
# Oh no, none of the p-values are significant!!!! that's because we only have three 
# samples per group and a significant result is impossible. 
# this was intentional, to highlight the challenges that may arise using non-parametric stats.

In [19]:
ft2 = ecoli[ecoli.columns[11:]]
print(ft2.shape)
scaler = StandardScaler()
x = scaler.fit_transform(ft2+1) # scale all features to have unit mean and equal variance

(3602, 0)


ValueError: at least one array or dtype is required